# Create dissolved Pb concentration initial conditions

Based on water masses identified from the simulated temperature and salinity, described in supplementary text S1.

In [2]:
import xarray as xr
import numpy as np
import xarray as xr
import sys
sys.path.append('../paper-materials/')
from constants import imin, imax, jmin, jmax

%matplotlib inline

#### Files

In [3]:
# The Pb model configuration ANHA12 mask:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
mesh_bathy   = mesh['tmask'].values[0,:,:,:]
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = mesh['nav_lev'].values

In [4]:
# Load an ANHA12 temperature and salinity file at the start of time series:
df_gridT = xr.open_dataset(f'/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2002m01d05.nc')
temp = df_gridT['votemper'].values[0,:,:,:]
sal  = df_gridT['vosaline'].values[0,:,:,:]

#### Functions

In [5]:
# Function creates initial dissolved Pb concentrations based on the provided temperature and salinity
def create_IC(temp, sal):
    # Pb end-members:
    PML_Pb    = 4e-12   # Polar Mixed Layer Pb
    AW_Pb     = 30e-12  # Atlantic water Pb
    AO_Pb     = 6e-12   # Arctic water Pb
    AOAW_Pb   = 25e-12  # Arctic Atlantic water mixture WGC
    
    PML_temp  = -0.2    # Polar mixed layer temperature
    AW_sal    = 35.2    # Atlantic Water salinity
    AW_temp   = 5.2     # Atlantic Water temperature
    
    # Linear interpolate endpoints between PML and ACW:
    end_temp = [PML_temp, AW_temp] 
    end_dPb = [PML_Pb, AW_Pb]
    dPb_IC  = np.interp(temp, end_temp, end_dPb)
    
    # Labrador Sea deep atlantic water inflow:
    end_sal = [34.86, 34.875]
    end_dPb = [AOAW_Pb, AW_Pb]
    dPb_IC[(sal > 34.86) & (temp > 2.0)] = np.interp(sal[(sal > 34.86) & (temp > 2.0)], end_sal, end_dPb)
    
    # Baffin Bay subsurface WGC increase
    end_sal = [34.0, AW_sal]
    end_dPb = [AO_Pb, AW_Pb]
    dPb_IC[:,:imax-500,:][(sal[:,:imax-500,:] > 33.7) & (sal[:,:imax-500,:] < 34.4) & (temp[:,:imax-500,:] < 0.4)] = \
        np.interp(sal[:,:imax-500,:][(sal[:,:imax-500,:] > 33.7) & (sal[:,:imax-500,:] < 34.4) & (temp[:,:imax-500,:] < 0.4)], \
                  end_sal, end_dPb)
    
    dPb_IC[np.isnan(dPb_IC)] = 3e-12
    
    return dPb_IC

In [6]:
# Save the initial conditions to a netCDF file
def save_file(ini_pb):    
    ini_pb_reshape = np.empty((1,50,2400,1632))
    ini_pb_reshape[0,:,:,:] = ini_pb
    
    file_write = xr.Dataset(
        {'dPb': (("time_counter","deptht","y","x"), ini_pb_reshape)}, 
        coords = {
            "time_counter": np.zeros(1),
            "deptht": np.zeros(50),
            "y": np.zeros(2400),
            "x": np.zeros(1632),
        },
        attrs = {
            "long_name":'Dissolved Pb concentration IC',
            "units":"kmol/m3"
        },
    )
    file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/Pb-tuning-202302/Pb-ini-20230214.nc', unlimited_dims='time_counter')
    return

#### Calculate

In [10]:
overall_IC = create_IC(temp, sal)

#### Save file

In [28]:
# Check no zeros, NaNs etc.
print(sum(sum(sum(np.isnan(overall_IC)))))
print(sum(sum(sum(overall_IC==0))))
print(np.amin(overall_IC), np.amax(overall_IC), np.mean(overall_IC))
print(overall_IC.shape)

0
0
3e-12 3e-11 8.676769836818529e-12
(50, 2400, 1632)


In [29]:
save_file(overall_IC)